In [1]:
import numpy as np
import pandas as pd
import math
from siuba import *
from calitp import *
import intake
import data_prep

import shared_utils
from plotnine import *
import altair as alt
import altair_saver
from shared_utils import altair_utils 

pd.options.display.float_format = "{:.2f}".format
pd.options.display.max_rows = 250

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
df_5311 = data_prep.load_grantprojects()
vehicles = data_prep.load_cleaned_vehiclesdata()
organizations = data_prep.load_cleaned_organizations_data()

/home/jovyan/data-analyses/5311/data_prep.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

# Merge NTD with GTFS  --> vehicle_gtfs
<b> NOTE</b>:
I had to manually add NTD ID to the following agencies in the original CSV file called "cleaned organizations.csv"

* Butte County Association of Governments	90208
* City of Dixon	9402-91041
* City of Fairfield	90092
* City of Ridgecrest	9R02-91006
* City of Wasco	9R02-99426
* Glenn County	9R02-91088
* Mariposa County	9R02-91082
* Modoc Transportation Authority 	9R02-91008
* Palo Verde Valley Transit Agency	9R02-99454
* San Benito County Local Transportation Authority	9R02-91009
* San Joaquin Regional Transit District	90012
* Tuolumne County Transit Agency	9402-035


In [3]:
#merging the 2 datasets together
vehicles_gtfs = pd.merge(vehicles, organizations,  how='left', on=['ntd_id'], indicator=True)

In [4]:
vehicle_agencies = vehicles_gtfs['agency'].drop_duplicates().tolist()

In [5]:
vehicles_gtfs['_merge'].value_counts()

both          192
left_only      26
right_only      0
Name: _merge, dtype: int64

In [6]:
vehicles_gtfs = vehicles_gtfs.drop(columns = ['_merge'])

# Merging -> Vehicle & GTFS with Black Cat.

In [7]:
#left merge, Black Cat on the left
Test1 = pd.merge(df_5311, vehicles_gtfs,  how='left', left_on=['organization_name'], right_on=['name'], indicator=True)

In [8]:
len(Test1)

798

In [9]:
# Counting left only and both
Test1['_merge'].value_counts()

both          495
left_only     303
right_only      0
Name: _merge, dtype: int64

In [10]:
#finding the agencies that are in left only.
Left_only = Test1[(Test1._merge.str.contains("left_only", case= False))] 
Left_orgs = Left_only['organization_name'].drop_duplicates().tolist()

In [11]:
# Filter out left only matches
m2 = Test1[~Test1.organization_name.isin(Left_orgs)]

In [12]:
#check that filter worked ok
m2.shape

(495, 45)

In [13]:
m2.sample(3)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,name,itp_id,gtfs_schedule_status,_merge
68,2016,Section 5311,CA-2017-025 | 0017000158,2016,Imperial County Transportation Commission,BCG0000186,Operating Assistance,300902,64BO17-00402,164302.00,164302.00,164302.00,0.00,0,Open,Imperial County Transportation Commission,90226,Full Reporter,46.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,46.00,4.23,142255.12,NaN,0.00,16.00,0.00,3.00,0.00,27.00,0.00,32.00,0.00,27.00,59.00,Imperial County Transportation Commission,138.00,"needed,needed,needed,needed",both
201,2017,Section 5311,CA-2017-168 | 0017000259,2017,Imperial County Transportation Commission,BCG0001230,Operating Assistance Sliding Scale,300902,64BO17-00577,280178.00,280178.00,280178.00,0.00,0,Open,Imperial County Transportation Commission,90226,Full Reporter,46.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,46.00,4.23,142255.12,NaN,0.00,16.00,0.00,3.00,0.00,27.00,0.00,32.00,0.00,27.00,59.00,Imperial County Transportation Commission,138.00,"needed,needed,needed,needed",both
745,2021,Section 5311,TBD | 0021000227,2021,Morongo Basin Transit Authority,BCG0002839,Operating Assistance Sliding Scale,300902,NaN,437331.00,0.00,0.00,437331.00,0,Open,Morongo Basin Transit Authority,9R02-91090,Rural Reporter,44.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,44.00,4.12,0.00,NaN,0.00,18.00,0.00,6.00,0.00,20.00,0.00,36.00,0.00,20.00,56.00,Morongo Basin Transit Authority,212.00,"ok,research",both


### Failed Matches, crosswalk manually

In [14]:
#making a data frame with only failed merges out out of original Black Cat
fail = df_5311[df_5311.organization_name.isin(Left_orgs)]

In [15]:
fail.shape

(303, 15)

In [16]:
#crosswalk dictionary for replacing organization name
crosswalk = {'City of Chowchilla ': 'City of Chowchilla, dba: Chowchilla Area Transit ',
 'City of Dinuba ':  'City of Dinuba',
 'Modoc Transportation Agency': 'Modoc Transportation Agency',
 'Butte County Association of Governments/ Butte Regional Transit': 'Butte County Association of Governments',
 'Calaveras County Public Works':  'Calaveras Transit Agency',
 'City of Escalon ':  'City of Escalon, dba: eTrans',
 'County of Mariposa':  'Mariposa County Transit, dba: Mari-Go',
 'County of Shasta Department of Public Works':  'County of Shasta Department of Public Works',
 'County of Siskiyou': 'County of Siskiyou, dba: Siskiyou County Transit',
 'County of Tulare': 'Tulare County Area Transit',
 'Eureka Transit Service':  'City of Eureka, dba: Eureka Transit Service',
 'Kern Regional Transit':  'Kern Regional Transit',
 'Livermore Amador Valley Transit Authority':  'Livermore / Amador Valley Transit Authority',
 'Placer County Public Works (TART & PCT)': 'County of Placer, dba: Placer County Department of Public Works',
 'Plumas County Transportation Commission': 'Plumas County Transportation Commission',
 'San Luis Obispo Regional Transit Authority':  'San Luis Obispo Regional Transit Authority',
 'Sonoma County Transit':  'County of Sonoma, dba: Sonoma County Transit',
 'Sunline Transit Agency':  'SunLine Transit Agency',
 'Tehama County Transit Agency': 'Tehama County',
 'Trinity County Department of Transportation ':  'Trinity County',
 'Tuolumne County Transit Agency (TCTA)':  'Tuolumne County Transit',
 'Amador Transit':  'Amador Regional Transit System',
 'City of Corcoran - Corcoran Area Transit':  'City of Corcoran, dba: Corcoran Area Transit',
 'Yosemite Area Regional Transportation System ':  'Yosemite Area Regional Transportation System',
 'County Connection (Central Contra Costa Transit Authority)': 'Central Contra Costa Transit Authority, dba: COUNTY CONNECTION',
 'Calaveras Transit Agency ': 'Calaveras Transit Agency'}

In [17]:
#failed merges
fail['organization_name'].replace(crosswalk, inplace= True)

/opt/conda/lib/python3.9/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
#Merging the failed organizations to vehicles 
Test2 = pd.merge(fail, vehicles_gtfs,  how='left', left_on=['organization_name'], right_on=['agency'], indicator=True)

In [19]:
Test2['_merge'].value_counts()

both          248
left_only      55
right_only      0
Name: _merge, dtype: int64

In [20]:
#finding the agencies that are in left only...make sure these are the ones we aren't sure have any matches.
Left_only = Test2[(Test2._merge.str.contains("left_only", case= False))]
#find failed agencies 

Left_only_orgs= Left_only['organization_name'].drop_duplicates().tolist()
Left_only_orgs

['County of Los Angeles - Department of Public Works',
 'County of Nevada Public Works, Transit Services Division',
 'County of Sacramento Department of Transportation',
 'Glenn County Transportation Commission',
 'Stanislaus County Public Works - Transit Division',
 'Alpine County Community Development',
 'Fresno Council of Governments',
 'Klamath Trinity Non-Emergency Transportation\u200b',
 'Greyhound Lines, Inc.']

In [21]:
#deleting Klamath off since it has GTFS, we only want agencies that do not appear in both GTFS & NTD 
L_only_2_orgs = ['County of Los Angeles - Department of Public Works',
 'County of Nevada Public Works, Transit Services Division',
 'County of Sacramento Department of Transportation',
 'Glenn County Transportation Commission',
 'Stanislaus County Public Works - Transit Division',
 'Alpine County Community Development',
 'Fresno Council of Governments',
 'Greyhound Lines, Inc.']

In [22]:
#appending failed matches to the first data frame
BC_GTFS_NTD = m2.append(Test2, ignore_index=True)

In [23]:
#checking that rows match up
BC_GTFS_NTD.shape

(798, 45)

In [24]:
#checking that organizations are here
BC_GTFS_NTD['organization_name'].nunique()

87

In [25]:
#drop  merge
BC_GTFS_NTD = BC_GTFS_NTD.drop(columns=['_merge'])

### Flag agencies that appear in Black Cat Only

In [26]:
def BC_only(row):
    if row.organization_name in Left_only_orgs:
        return '1'
    else: 
        return '0'  

In [27]:
BC_GTFS_NTD["Is_Agency_In_BC_Only_1_means_Yes"] = BC_GTFS_NTD.apply(lambda x: BC_only(x), axis=1)

In [28]:
#Checking again 
BC_GTFS_NTD[(BC_GTFS_NTD.organization_name.str.contains("greyhound", case= False))].head(1)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,name,itp_id,gtfs_schedule_status,Is_Agency_In_BC_Only_1_means_Yes
578,2017,Section 5311(f),CA-2017-168 | 0017000259-F,2017,"Greyhound Lines, Inc.",BCG0000173,Operating Assistance - Route 1 Service from Ar...,300902,64CO17-00447,161293.00,247887.00,161293.00,0.00,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


### Replacing Klamath
* Klamath has GTFS but no NTD records

In [29]:
BC_GTFS_NTD.loc[(BC_GTFS_NTD['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "itp_id"] = "436"
BC_GTFS_NTD.loc[(BC_GTFS_NTD['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "gtfs_schedule_status"] = "needed"

In [30]:
#Checking again 
BC_GTFS_NTD[(BC_GTFS_NTD.organization_name.str.contains("Klamath", case= False))].head(1)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,name,itp_id,gtfs_schedule_status,Is_Agency_In_BC_Only_1_means_Yes
544,2017,5311(f) Cont,TBD,2017,Klamath Trinity Non-Emergency Transportation​,BCG0000168,Operating Assistance,300902,64CO17-00467,8344.00,8344.00,8344.00,0.00,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,436,needed,1


### Getting ok/needed/research out of original GTFS data frame

In [31]:
#look at original GTFS status for all the different varities...
BC_GTFS_NTD['gtfs_schedule_status'].unique()

array(['ok,needed,needed',
       'needed,needed,ok,ok,needed,ok,ok,needed,needed,needed,needed,needed,ok,ok,ok,needed,ok,needed,needed,needed,needed,needed',
       'ok', 'needed,ok,needed,ok', 'needed,ok,ok,ok',
       'need long-term solution,needed', 'ok,needed', 'needed',
       'ok,needed,ok', 'needed,ok', 'need long-term solution', nan,
       'ok,needed,ok,ok,ok,needed,needed,needed,needed',
       'ok,needed,needed,needed,needed', 'needed,needed,needed,needed',
       'needed,ok,needed,needed,needed,needed,needed,needed,needed',
       'ok,needed,needed,needed,needed,needed',
       'needed,needed,needed,needed,research,needed', 'ok,research',
       'ok,needed,needed,ok,needed,needed,research,needed',
       'ok,ok,ok,needed,needed', 'needed,ok,needed',
       'ok,ok,ok,ok,needed,needed',
       'ok,needed,needed,needed,research,needed',
       'ok,ok,ok,ok,ok,ok,needed,needed', 'needed,needed,ok,ok',
       'needed,needed,needed', 'needed,ok,needed,needed', 'ok,ok',
       '

In [32]:
#create a new column
temp = BC_GTFS_NTD.gtfs_schedule_status.fillna("None")
BC_GTFS_NTD['GTFS_schedule_status_use'] = np.where(temp.str.contains("None"),"None",
                   np.where(temp.str.contains("ok"), "Ok",
                   np.where(temp.str.contains("long"), "Long-term solution needed",
                   np.where(temp.str.contains("research"), "Research", "Needed"))))


### Rating Fleet Size
* Using the aggregated df below, I did aggregate1['total_vehicles'].describe() and looked at the different percentiles to choose small, med, large, and no info

In [33]:
#First grabbing only one row for each agency.
Fleet_size = BC_GTFS_NTD.groupby(['organization_name',]).agg({'total_vehicles':'max'}).reindex()

In [34]:
type(Fleet_size)

pandas.core.frame.DataFrame

In [35]:
Fleet_size['total_vehicles'].describe()

count     78.00
mean      89.69
std      189.60
min        1.00
25%        9.25
50%       24.50
75%       80.25
max     1114.00
Name: total_vehicles, dtype: float64

In [36]:
#Get percentiles in a variable
p75 = Fleet_size.total_vehicles.quantile(0.75).astype(float)
p25 = Fleet_size.total_vehicles.quantile(0.25).astype(float)
p50 = Fleet_size.total_vehicles.quantile(0.50).astype(float)


In [37]:
p25

9.25

In [38]:
#Function
def fleet_size (row):
            if ((row.total_vehicles > 0) and (row.total_vehicles < p25)):
                return "Small"
            elif ((row.total_vehicles > p25) and (row.total_vehicles < p75)):
                return "Medium"
            elif ((row.total_vehicles > p50) and (row.total_vehicles > p75 )):
                return "Large"
            else:
                return "No Info"

In [39]:
BC_GTFS_NTD["fleet_size"] = BC_GTFS_NTD.apply(lambda x: fleet_size(x), axis=1)

In [40]:
#Checking again 
BC_GTFS_NTD[(BC_GTFS_NTD.organization_name.str.contains("Shafter", case= False))].head(2)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,name,itp_id,gtfs_schedule_status,Is_Agency_In_BC_Only_1_means_Yes,GTFS_schedule_status_use,fleet_size
34,2016,Section 5311,CA-2017-025 | 0017000158,2016,City of Shafter,BCG0000177,Operating Assistance,300902,64BO17-00383,55562.00,55562.00,55562.00,0.00,0,Open,"City of Shafter, dba: Shafter Dial-A-Ride",9R02-91120,Rural Reporter,7.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,9.00,7.50,0.00,NaN,0.00,0.00,0.00,0.00,0.00,9.00,0.00,0.00,0.00,9.00,9.00,City of Shafter,303.00,needed,0,Needed,Small
116,2017,Section 5311,CA-2017-168 | 0017000259,2017,City of Shafter,BCG0001059,Operating Assistance Sliding Scale,300902,64BO17-00547,56127.00,56127.00,56127.00,0.00,0,Open,"City of Shafter, dba: Shafter Dial-A-Ride",9R02-91120,Rural Reporter,7.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,9.00,7.50,0.00,NaN,0.00,0.00,0.00,0.00,0.00,9.00,0.00,0.00,0.00,9.00,9.00,City of Shafter,303.00,needed,0,Needed,Small


In [41]:
BC_GTFS_NTD.fleet_size.value_counts()

Medium     385
Large      200
Small      158
No Info     55
Name: fleet_size, dtype: int64

### Final Cleaning

In [42]:
#delete old columns
BC_GTFS_NTD2 = BC_GTFS_NTD.drop(columns=['gtfs_schedule_status','name','agency'])

#rename
BC_GTFS_NTD2 = BC_GTFS_NTD2.rename(columns = {'GTFS_schedule_status_use':'GTFS'})

In [43]:
#get agencies without any data to show up 
show_up = ['reporter_type']
for i in show_up:
    BC_GTFS_NTD2[i] = BC_GTFS_NTD2[i].fillna('None')

In [44]:
#change itp id to be float.
BC_GTFS_NTD2['itp_id'] = BC_GTFS_NTD2['itp_id'].fillna(0)
BC_GTFS_NTD2.loc[(BC_GTFS_NTD['itp_id'] == '436'), "itp_id"] = 436

### Parquet

In [45]:
#Parquet
BC_GTFS_NTD2.to_parquet("BC_GTFS_NTD.parquet")
#Adding to GCS (just in case)
#BC_GTFS_NTD2.to_parquet("gs://calitp-analytics-data/data-analyses/5311 /BC_GTFS_NTD.parquet")
#BC_GTFS_NTD2.to_csv("gs://calitp-analytics-data/data-analyses/5311 /BC_GTFS_NTD.csv")

### Script Testing for FULL merged data frame
* Full = all the years, UPIN, contract, etc

In [46]:
TEST_crosswalk = {'City of Chowchilla ': 'City of Chowchilla, dba: Chowchilla Area Transit ',
     'City of Dinuba ':  'City of Dinuba',
     'Modoc Transportation Agency': 'Modoc Transportation Agency',
     'Butte County Association of Governments/ Butte Regional Transit': 'Butte County Association of Governments',
     'Calaveras County Public Works':  'Calaveras Transit Agency',
     'City of Escalon ':  'City of Escalon, dba: eTrans',
     'County of Mariposa':  'Mariposa County Transit, dba: Mari-Go',
     'County of Shasta Department of Public Works':  'County of Shasta Department of Public Works',
     'County of Siskiyou': 'County of Siskiyou, dba: Siskiyou County Transit',
     'County of Tulare': 'Tulare County Area Transit',
     'Eureka Transit Service':  'City of Eureka, dba: Eureka Transit Service',
     'Kern Regional Transit':  'Kern Regional Transit',
     'Livermore Amador Valley Transit Authority':  'Livermore / Amador Valley Transit Authority',
     'Placer County Public Works (TART & PCT)': 'County of Placer, dba: Placer County Department of Public Works',
     'Plumas County Transportation Commission': 'Plumas County Transportation Commission',
     'San Luis Obispo Regional Transit Authority':  'San Luis Obispo Regional Transit Authority',
     'Sonoma County Transit':  'County of Sonoma, dba: Sonoma County Transit',
     'Sunline Transit Agency':  'SunLine Transit Agency',
     'Tehama County Transit Agency': 'Tehama County',
     'Trinity County Department of Transportation ':  'Trinity County',
     'Tuolumne County Transit Agency (TCTA)':  'Tuolumne County Transit',
     'Amador Transit':  'Amador Regional Transit System',
     'City of Corcoran - Corcoran Area Transit':  'City of Corcoran, dba: Corcoran Area Transit',
     'Yosemite Area Regional Transportation System ':  'Yosemite Area Regional Transportation System',
     'County Connection (Central Contra Costa Transit Authority)': 'Central Contra Costa Transit Authority, dba: COUNTY CONNECTION',
     'Calaveras Transit Agency ': 'Calaveras Transit Agency'}

#Agencies that ONLY appear in Black Cat
BC_Agency_List = ['County of Los Angeles - Department of Public Works',
 'County of Nevada Public Works, Transit Services Division',
 'County of Sacramento Department of Transportation',
 'Glenn County Transportation Commission',
 'Stanislaus County Public Works - Transit Division',
 'Alpine County Community Development',
 'Fresno Council of Governments',
 'Greyhound Lines, Inc.']

In [47]:
#NOTE TO SELF: once finalized, change TEST-crosswalk back to just crosswalk.
def merged_dataframe():
    ### MERGING ###
    #call the 3 data frames we have to join.
    df_5311 = data_prep.load_grantprojects()
    vehicles = data_prep.load_cleaned_vehiclesdata()
    organizations = data_prep.load_cleaned_organizations_data()
    #merge vehicles from NTD & GTFS
    vehicles_gtfs = pd.merge(vehicles, organizations,  how='left', on=['ntd_id'])
    #left merge, Black Cat on the left and vehicle_gtfs on the right. 
    m1 = (pd.merge(df_5311, vehicles_gtfs,  how='left', left_on=['organization_name'], 
                      right_on=['name'], indicator=True)
            )
    
    #Filter out for left only matches, make it into a list
    Left_only = m1[(m1._merge.str.contains("left_only", case= False))] 
    Left_orgs = Left_only['organization_name'].drop_duplicates().tolist()
    #Delete  left only matches from original df 
    m2 = m1[~m1.organization_name.isin(Left_orgs)]
    #making a data frame with only failed merges out out of original Black Cat
    fail = df_5311[df_5311.organization_name.isin(Left_orgs)]
    #replacing organization names from Black Cat with agency names from NTD Vehicles.
    fail['organization_name'].replace(crosswalk, inplace= True)
    #Merging the failed organizations to vehicles 
    Test = pd.merge(fail, vehicles_gtfs,  how='left', left_on=['organization_name'], right_on=['agency'])
    #appending failed matches to the first data frame
    BC_GTFS_NTD = m2.append(Test, ignore_index=True)
    
    ### METRIC TO SHOW BLACK CAT ONLY AGENCIES ###
    #Function
    def BC_only(row):
        if row.organization_name in BC_Agency_List:
            return '1'
        else: 
            return '0'  
    BC_GTFS_NTD["Is_Agency_In_BC_Only_1_means_Yes"] = BC_GTFS_NTD.apply(lambda x: BC_only(x), axis=1)
    
    #Replace GTFS and cal itp for Klamath. 
    #Klamath does not appear in NTD data so we missed it when we merged NTD & Cal ITP on NTD ID
    BC_GTFS_NTD.loc[(BC_GTFS_NTD['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "itp_id"] = "436"
    BC_GTFS_NTD.loc[(BC_GTFS_NTD['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "gtfs_schedule_status"] = "needed"
    
    #get a more definitive GTFS status: ok, needed, long term solution needed, research
    temp = BC_GTFS_NTD.gtfs_schedule_status.fillna("None")
    BC_GTFS_NTD['GTFS_schedule_status_use'] = np.where(temp.str.contains("None"),"None",
                   np.where(temp.str.contains("ok"), "Ok",
                   np.where(temp.str.contains("long"), "Long-term solution needed",
                   np.where(temp.str.contains("research"), "Research", "Needed"))))
    
    ###FLEET SIZE METRIC ### 
    #First grabbing only one row for each agency into a new data frame 
    Fleet_size = BC_GTFS_NTD.groupby(['organization_name',]).agg({'total_vehicles':'max'}).reindex()
    #Get percentiles in objects for total vehicle.
    p75 = Fleet_size.total_vehicles.quantile(0.75).astype(int)
    p25 = Fleet_size.total_vehicles.quantile(0.25).astype(int)
    p50 = Fleet_size.total_vehicles.quantile(0.50).astype(int)
    #Function for fleet size
    def fleet_size (row):
        if ((row.total_vehicles > 0) and (row.total_vehicles < p25)):
            return "Small"
        elif ((row.total_vehicles > p25) and (row.total_vehicles < p75)):
            return "Medium"
        elif ((row.total_vehicles > p50) and (row.total_vehicles > p75 )):
               return "Large"
        else:
            return "No Info"
    BC_GTFS_NTD["fleet_size"] = BC_GTFS_NTD.apply(lambda x: fleet_size(x), axis=1)
    
    ### FINAL CLEAN UP ###
    #delete old columns
    BC_GTFS_NTD = BC_GTFS_NTD.drop(columns=['gtfs_schedule_status','name','agency'])
    #rename
    BC_GTFS_NTD = BC_GTFS_NTD.rename(columns = {'GTFS_schedule_status_use':'GTFS'})
    #get agencies without any data to show up when grouping
    show_up = ['reporter_type']
    for i in show_up:
        BC_GTFS_NTD[i] = BC_GTFS_NTD[i].fillna('None')
    #change itp id to be floats & 0 so parquet will work
    BC_GTFS_NTD['itp_id'] = BC_GTFS_NTD['itp_id'].fillna(0)
    BC_GTFS_NTD.loc[(BC_GTFS_NTD['itp_id'] == '436'), "itp_id"] = 436
    #save into parquet
    BC_GTFS_NTD2.to_parquet("BC_GTFS_NTD.parquet")
    return BC_GTFS_NTD 

In [48]:
test = merged_dataframe()

test.sample(5)

/home/jovyan/data-analyses/5311/data_prep.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,itp_id,_merge,Is_Agency_In_BC_Only_1_means_Yes,GTFS,fleet_size
787,2021,Section 5311(f),TBD | 0021000227-F,2021,County of Shasta Department of Public Works,BCG0002837,Operating Assistance Sliding Scale,300902,NaN,236755.00,0.00,0.00,236755.00,0,Open,9R02-99438,Asset Subrecipient,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,6.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,1.00,0,NaN,0,Ok,Small
704,2019,Section 5311(f),CA-2018-114 | 0019000012,2019,SunLine Transit Agency,BCG0001454,Operating Assistance Sliding Scale,300902,64CO19-00908,186051.00,186051.00,186051.00,0.00,0,Open,90079,Full Reporter,125.00,49.00,4.00,0.00,0.00,0.00,0.00,0.00,178.00,3.47,114625.36,NaN,1.00,85.00,0.00,48.00,0.00,44.00,2.00,170.00,0.00,44.00,216.00,327.00,NaN,0,Ok,Large
721,2020,Section 5311,CA-2021-030 | 0020000131,2020,County of Shasta Department of Public Works,BCG0002395,Operating Assistance Sliding Scale,300902,64BO20-01419,160881.00,360153.00,360153.00,-199272.00,0,Open,9R02-99438,Asset Subrecipient,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,6.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,1.00,0,NaN,0,Ok,Small
625,2018,Section 5311,CA-2018-114 | 0019000012,2018,Kern Regional Transit,BCG0001131,Operating Assistance Sliding Scale,300902,64BO19-00856,984248.00,984248.00,984248.00,0.00,0,Open,9R02-91059,Rural Reporter,60.00,10.00,5.00,12.00,0.00,0.00,0.00,0.00,87.00,9.26,0.00,NaN,0.00,26.00,0.00,1.00,0.00,60.00,0.00,52.00,0.00,60.00,112.00,146.00,NaN,0,Ok,Large
575,2017,Section 5311,CA-2017-168 | 0017000259,2017,Trinity County,BCG0001335,Purchase Replacement < 30 Ft Bus,111204,64BC17-00509,74378.00,74378.00,74378.00,0.00,0,Open,9R02-91035,Rural Reporter,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.00,2.58,0.00,NaN,0.00,0.00,0.00,1.00,0.00,7.00,0.00,0.00,0.00,7.00,7.00,344.00,NaN,0,Ok,Small
